### Testing, scrap, for how to implement ANOVA for belief state partitions, 

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)
import utils.behavioral_utils as behavioral_utils
import utils.spike_utils as spike_utils
import utils.io_utils as io_utils
import os
from distutils.util import strtobool

import argparse
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
from scripts.anova_analysis.anova_configs import *
from constants.behavioral_constants import *
from constants.decoding_constants import *

from scipy import stats
from scripts.anova_analysis.run_anova import load_data
import utils.anova_utils as anova_utils


In [3]:
FEATS_PATH = "/data/patrick_res/sessions/{sub}/feats_at_least_3blocks.pickle"
SESSIONS_PATH = "/data/patrick_res/sessions/{sub}/valid_sessions.pickle"

args = argparse.Namespace(
    **AnovaConfigs()._asdict()
)
args.trial_event = "FeedbackOnsetLong"
args.conditions = ["BeliefConf", "BeliefPartition"]
args.time_range = [0, 1500]
args.feat = "SQUARE"
args.beh_filters = {"Response": "Correct", "Choice": "Chose"}

feat_sessions = pd.read_pickle(FEATS_PATH.format(sub=args.subject))
valid_sess = pd.read_pickle(SESSIONS_PATH.format(sub=args.subject))
row = feat_sessions[feat_sessions.feat == args.feat].iloc[0]
args.sessions = valid_sess[valid_sess.session_name.isin(row.sessions)]
args.trial_interval = get_trial_interval(args.trial_event)  

In [4]:
data = load_data("20180802", args)

In [5]:
all_conds = ["TimeBins"] + args.conditions
df = anova_utils.anova_factors(data, all_conds)

In [6]:
df.columns

Index(['TrialNumber', 'UnitID', 'TimeBins', 'SpikeCounts', 'FiringRate',
       'PseudoUnitID', 'Time', 'BlockNumber', 'TrialAfterRuleChange',
       'TaskInterrupt', 'ConditionNumber', 'Response', 'ItemChosen',
       'TrialType', 'CurrentRule', 'LastRule', 'ShiftType', 'Perseveration',
       'TrialStart', 'FixCrossOn', 'FixationOnCross', 'FixRequireEnd',
       'StimOnset', 'FeedbackEncode', 'FeedbackOnset', 'TrialEnd',
       'Item0_xPos', 'Item0_yPos', 'Item1_xPos', 'Item1_yPos', 'Item2_xPos',
       'Item2_yPos', 'Item3_xPos', 'Item3_yPos', 'Item0Shape', 'Item0Color',
       'Item0Pattern', 'Item1Shape', 'Item1Color', 'Item1Pattern',
       'Item2Shape', 'Item2Color', 'Item2Pattern', 'Item3Shape', 'Item3Color',
       'Item3Pattern', 'Color', 'Shape', 'Pattern', 'CIRCLEProb', 'SQUAREProb',
       'STARProb', 'TRIANGLEProb', 'CYANProb', 'GREENProb', 'MAGENTAProb',
       'YELLOWProb', 'ESCHERProb', 'POLKADOTProb', 'RIPPLEProb', 'SWIRLProb',
       'BeliefStateValue', 'BeliefStateV

In [7]:
unit_vars = df.groupby("PseudoUnitID").apply(lambda x: anova_utils.calc_unit_var(x, all_conds)).reset_index()


In [8]:
unit_vars.columns

Index(['PseudoUnitID', 'total_var', 'x_TimeBins_fracvar',
       'x_BeliefConf_fracvar', 'x_BeliefPartition_fracvar',
       'x_TimeBinsBeliefConf_fracvar', 'x_TimeBinsBeliefPartition_fracvar',
       'x_BeliefConfBeliefPartition_fracvar',
       'x_TimeBinsBeliefConfBeliefPartition_fracvar', 'residual_fracvar',
       'sum_fracvar'],
      dtype='object')

In [9]:
combs = anova_utils.get_combs_of_conds(args.conditions)
for comb in combs: 
    combined_cond_str = "".join(comb)
    unit_vars[f"x_{combined_cond_str}_comb_time_fracvar"] = unit_vars[f"x_{combined_cond_str}_fracvar"] + unit_vars[f"x_TimeBins{combined_cond_str}_fracvar"]

In [10]:
unit_vars[f"x_BeliefPref_comb_time_fracvar"] = unit_vars[f"x_BeliefPartition_comb_time_fracvar"] - unit_vars[f"x_BeliefConf_comb_time_fracvar"]


In [11]:
unit_vars

,PseudoUnitID,total_var,x_TimeBins_fracvar,x_BeliefConf_fracvar,x_BeliefPartition_fracvar,x_TimeBinsBeliefConf_fracvar,x_TimeBinsBeliefPartition_fracvar,x_BeliefConfBeliefPartition_fracvar,x_TimeBinsBeliefConfBeliefPartition_fracvar,residual_fracvar,sum_fracvar,x_BeliefConf_comb_time_fracvar,x_BeliefPartition_comb_time_fracvar,x_BeliefConfBeliefPartition_comb_time_fracvar,x_BeliefPref_comb_time_fracvar
0,2018080200,91.450562,0.063304,5.972352e-05,0.023739,0.000911,0.001964,5.972352e-05,0.001091,0.911052,1.002181,0.000971,0.025703,0.001150,0.024732
1,2018080201,7.053058,0.006225,2.668551e-03,0.004169,0.004794,0.006604,2.668551e-03,0.012800,0.985671,1.025600,0.007463,0.010773,0.015468,0.003311
2,2018080202,24.086206,0.051266,5.014880e-04,0.000989,0.003276,0.007295,5.014880e-04,0.004780,0.940952,1.009561,0.003777,0.008284,0.005282,0.004507
3,2018080203,19.286836,0.036237,5.070993e-04,0.001246,0.003970,0.005919,5.070993e-04,0.005491,0.957106,1.010982,0.004477,0.007164,0.005998,0.002688
4,2018080204,39.342802,0.205823,1.851591e-04,0.000186,0.001425,0.006256,1.851591e-04,0.001981,0.787920,1.003962,0.001611,0.006442,0.002166,0.004831
5,2018080205,32.222398,0.031433,4.900148e-05,0.000831,0.003920,0.005848,4.900148e-05,0.004067,0.961938,1.008134,0.003969,0.006678,0.004116,0.002709
6,2018080206,18.771418,0.082717,3.741276e-04,0.000713,0.003624,0.012792,3.741276e-04,0.004746,0.904152,1.009493,0.003998,0.013505,0.005121,0.009507
7,2018080207,0.886825,0.023017,3.314502e-05,0.001188,0.001873,0.003893,3.314502e-05,0.001973,0.971935,1.003946,0.001907,0.005081,0.002006,0.003174
8,2018080208,8.351690,0.047823,5.898266e-05,0.001511,0.003198,0.004147,5.898266e-05,0.003375,0.946578,1.006751,0.003257,0.005658,0.003434,0.002400
9,2018080209,141.219100,0.124385,1.362562e-02,0.014407,0.018282,0.020199,1.362562e-02,0.059159,0.854635,1.118318,0.031908,0.034606,0.072785,0.002699


In [3]:
feat = args.feat
beh = behavioral_utils.load_behavior_from_args("20180802", args)
beh = behavioral_utils.get_belief_partitions(beh, feat)
beh["Choice"] = beh.apply(lambda x: "Chose" if x[FEATURE_TO_DIM[feat]] == feat else "Not Chose", axis=1)
beh["FeatPreferred"] = beh["PreferredBelief"].apply(lambda x: "Preferred" if x == feat else "Not Preferred")


beh = behavioral_utils.filter_behavior(beh, args.beh_filters)

In [5]:
beh.BeliefConf.unique()

array(['Low'], dtype=object)

In [12]:
res = pd.read_pickle("/data/patrick_res/anova/SA_FeedbackOnsetLong_BeliefConf_BeliefPartition_Response_Correct_Choice_Chose/CIRCLE_.pickle")

In [14]:
res.sort_values(by="x_BeliefPref_comb_time_fracvar", ascending=False)

,PseudoUnitID,total_var,x_TimeBins_fracvar,x_BeliefConf_fracvar,x_BeliefPartition_fracvar,x_TimeBinsBeliefConf_fracvar,x_TimeBinsBeliefPartition_fracvar,x_BeliefConfBeliefPartition_fracvar,x_TimeBinsBeliefConfBeliefPartition_fracvar,residual_fracvar,sum_fracvar,x_BeliefConf_comb_time_fracvar,x_BeliefPartition_comb_time_fracvar,x_BeliefConfBeliefPartition_comb_time_fracvar,x_BeliefPref_comb_time_fracvar,feat
4,2018070904,5.683932,0.005472,0.001256,0.043969,0.003044,0.005010,0.001256,0.006811,0.946804,1.013623,0.004300,0.048979,0.008067,0.044680,CIRCLE
8,2018080608,37.659945,0.029912,0.009826,0.043345,0.014483,0.016819,0.009826,0.043962,0.919750,1.087924,0.024309,0.060164,0.053789,0.035854,CIRCLE
19,2018061519,39.291515,0.041281,0.001419,0.027316,0.009199,0.018994,0.001419,0.013457,0.913828,1.026913,0.010618,0.046310,0.014876,0.035691,CIRCLE
14,2018100514,22.071451,0.004611,0.000094,0.028561,0.003581,0.008316,0.000094,0.003861,0.958606,1.007723,0.003674,0.036877,0.003955,0.033202,CIRCLE
8,2018070908,6.783185,0.043466,0.000006,0.026918,0.015893,0.021995,0.000006,0.015911,0.907627,1.031822,0.015899,0.048913,0.015917,0.033014,CIRCLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,2018090713,41.099267,0.771531,0.000282,0.000303,0.001965,0.003050,0.000282,0.002810,0.225398,1.005620,0.002246,0.003353,0.003092,0.001106,CIRCLE
4,2018082204,163.622686,0.407132,0.003654,0.003785,0.017344,0.018210,0.003654,0.028307,0.574526,1.056615,0.020999,0.021996,0.031962,0.000997,CIRCLE
43,2018070943,50.222365,0.673018,0.002850,0.002854,0.007223,0.008154,0.002850,0.015773,0.318824,1.031546,0.010073,0.011008,0.018623,0.000935,CIRCLE
18,2018091918,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIRCLE
